### 1. ARDS 환자의 필요한 열(data)만 Chartevents에서 가져오기
    awk -F ',' 'NR==FNR{a[$1,$2]; next} FNR==1{print "SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM"; next} ($2,$3) in a {print $2","$3","$5","$6","$9","$10}' ~/project/MIMIC-III/ARDS.csv /data/PUBLIC_DATA/MIMIC-III/CHARTEVENTS.csv > ~/project/MIMIC-III/chartdata.csv

### 2. re-extract

In [46]:
import pandas as pd
import numpy as np

In [5]:
chartevents = pd.read_csv("~/project/MIMIC-III/chartdata.csv")
chartevents

/tmp/ipykernel_2171007/3414236497.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv("~/project/MIMIC-III/chartdata.csv")


,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM
0,165,170252,223761,2170-10-03 18:00:00,97.3,97.3
1,165,170252,225664,2170-10-03 18:00:00,174.0,174.0
2,165,170252,220277,2170-10-03 18:35:00,99.0,99.0
3,165,170252,220045,2170-10-03 18:36:00,73.0,73.0
4,165,170252,220210,2170-10-03 18:36:00,19.0,19.0
...,...,...,...,...,...,...
76592003,99776,136231,227341,2171-07-18 08:00:00,No,0
76592004,99776,136231,227342,2171-07-18 08:00:00,Yes,15
76592005,99776,136231,227343,2171-07-18 08:00:00,Bed rest,0
76592006,99776,136231,227344,2171-07-18 08:00:00,Yes,20


#### 2-1. 24H

In [12]:
df = chartevents

# 환자별 첫 24시간 이내의 데이터만 남기기
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).apply(lambda x: x[(x['CHARTTIME'] == x['CHARTTIME'].min()) | ((x['CHARTTIME'] > x['CHARTTIME'].min()) & (x['CHARTTIME'] <= x['CHARTTIME'].min() + pd.Timedelta(hours=24)))])
df1 = df1.reset_index(drop=True)

df1.to_csv("~/project/MIMIC-III/chartdata_24h.csv", index=False)
df1

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM
0,91,121205,742,2177-04-27 08:00:00,1,1.0
1,91,121205,1125,2177-04-27 08:00:00,MICU,NaN
2,91,121205,31,2177-04-27 09:00:00,Bedrest,NaN
3,91,121205,32,2177-04-27 09:00:00,Tolerated Well,NaN
4,91,121205,54,2177-04-27 09:00:00,2 Person Assist,NaN
...,...,...,...,...,...,...
8227714,99985,176670,227668,2181-01-30 00:00:00,3 rails up,NaN
8227715,99985,176670,227700,2181-01-30 00:00:00,Adequate lighting,NaN
8227716,99985,176670,224281,2181-01-30 00:31:00,WNL,NaN
8227717,99985,176670,224283,2181-01-30 00:31:00,Sharp,NaN


In [50]:
# VALUENUM이 NaN인 경우 데이터의 값이 숫자가 아닌 경우 의미
print("결측치 제거 전:", len(df1))
df2 = df1.dropna()
print("결측치 제거 후:", len(df2))

# VALUENUM에 들어간 문자 데이터 제거
df2 = df2[pd.to_numeric(df2['VALUENUM'], errors='coerce').notnull()]
print("문자 제거 후:", len(df2))

결측치 제거 전: 8227719
결측치 제거 후: 3870575
문자 제거 후: 3869685


#### 2-2. Mean 

In [65]:
# 환자별 첫 24시간 이내의 데이터의 Mean 구하기
df2.fillna(0, inplace=True)  # 결측치를 0으로 변경
df_mean = df2.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.mean(x.astype(float))})
df_mean

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,91,121205,87,15.000000
1,91,121205,184,3.666667
2,91,121205,198,14.666667
3,91,121205,211,107.400000
4,91,121205,454,6.000000
...,...,...,...,...
640955,99985,176670,227466,47.700000
640956,99985,176670,227467,1.100000
640957,99985,176670,227565,2200.000000
640958,99985,176670,227566,2800.000000


#### 2-3. Min

In [52]:
# 환자별 첫 24시간 이내의 데이터의 Min 구하기
df2.fillna(0, inplace=True)  # 결측치를 0으로 변경
df_min = df2.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.min(x.astype(float))})
df_min

,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,91,121205,87,15.0
1,91,121205,184,3.0
2,91,121205,198,14.0
3,91,121205,211,102.0
4,91,121205,454,6.0
...,...,...,...,...
640955,99985,176670,227466,47.7
640956,99985,176670,227467,1.1
640957,99985,176670,227565,2200.0
640958,99985,176670,227566,2800.0


#### 2-4. Max

In [53]:
# 환자별 첫 24시간 이내의 데이터의 Max 구하기
df2.fillna(0, inplace=True)  # 결측치를 0으로 변경
df_max = df2.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.max(x.astype(float))})
df_max

IOStream.flush timed out


,SUBJECT_ID,HADM_ID,ITEMID,VALUENUM
0,91,121205,87,15.0
1,91,121205,184,4.0
2,91,121205,198,15.0
3,91,121205,211,115.0
4,91,121205,454,6.0
...,...,...,...,...
640955,99985,176670,227466,47.7
640956,99985,176670,227467,1.1
640957,99985,176670,227565,2200.0
640958,99985,176670,227566,2800.0
